# Model Conversion for Supabase Edge Functions

This notebook converts your Keras model and pickle scalers to TensorFlow.js format and JSON.

## Steps:
1. Upload your files: `purair_model.keras`, `scaler_X.pkl`, `scaler_y.pkl`
2. Run all cells
3. Download the converted files
4. Upload them to Supabase Storage in the `models` bucket

In [ ]:
# Install required packages
!pip install tensorflowjs tensorflow scikit-learn -q

In [ ]:
import tensorflow as tf
import tensorflowjs as tfjs
import pickle
import json
import numpy as np
from google.colab import files
import os
import shutil

In [ ]:
# Upload your files
print("Please upload: purair_model.keras, scaler_X.pkl, scaler_y.pkl")
uploaded = files.upload()

In [ ]:
# Load the Keras model
print("Loading Keras model...")
model = tf.keras.models.load_model('purair_model.keras')
print("\nModel Summary:")
model.summary()

# Get input shape
input_shape = model.input_shape
print(f"\nInput shape: {input_shape}")

In [ ]:
# CRITICAL: Rebuild model with explicit input shape
# This ensures the model works properly in TensorFlow.js

print("Rebuilding model with explicit input shape...")

# Create new model with explicit input layer
new_model = tf.keras.Sequential()

# Add input layer with explicit shape [1, 3] for LSTM
# (1 timestep, 3 features: temperature, humidity, air_quality)
new_model.add(tf.keras.layers.InputLayer(input_shape=(1, 3), batch_size=1))

# Copy all layers except the first one if it's InputLayer
for i, layer in enumerate(model.layers):
    if i == 0 and isinstance(layer, tf.keras.layers.InputLayer):
        continue
    new_model.add(layer)

# Copy weights
new_model.set_weights(model.get_weights())

# Compile
new_model.compile(optimizer='adam', loss='mse')

model = new_model
print("\nRebuilt Model Summary:")
model.summary()

In [ ]:
# Test the model works
print("\nTesting model with sample input...")
sample_input = np.array([[[25.0, 60.0, 45.0]]])  # shape: (1, 1, 3)
prediction = model.predict(sample_input, verbose=0)
print(f"Sample prediction: {prediction[0][0]}")
print("Test successful!")

In [ ]:
# Convert model to TensorFlow.js format
print("\nConverting model to TensorFlow.js format...")
tfjs_output_dir = 'tfjs_model'

# Remove old directory if exists
if os.path.exists(tfjs_output_dir):
    shutil.rmtree(tfjs_output_dir)

# Convert
tfjs.converters.save_keras_model(model, tfjs_output_dir)
print(f"Model converted successfully to {tfjs_output_dir}")

# List generated files
print("\nGenerated files:")
for file in os.listdir(tfjs_output_dir):
    print(f"  - {file}")

In [ ]:
# Convert scaler_X.pkl to JSON
print("\nConverting scaler_X...")
with open('scaler_X.pkl', 'rb') as f:
    scaler_X = pickle.load(f)

scaler_X_dict = {
    'mean': scaler_X.mean_.tolist(),
    'scale': scaler_X.scale_.tolist()
}

with open('scaler_X.json', 'w') as f:
    json.dump(scaler_X_dict, f, indent=2)

print("scaler_X.json created")
print(f"  Mean: {scaler_X_dict['mean']}")
print(f"  Scale: {scaler_X_dict['scale']}")

In [ ]:
# Convert scaler_y.pkl to JSON
print("\nConverting scaler_y...")
with open('scaler_y.pkl', 'rb') as f:
    scaler_y = pickle.load(f)

scaler_y_dict = {
    'mean': scaler_y.mean_.tolist(),
    'scale': scaler_y.scale_.tolist()
}

with open('scaler_y.json', 'w') as f:
    json.dump(scaler_y_dict, f, indent=2)

print("scaler_y.json created")
print(f"  Mean: {scaler_y_dict['mean']}")
print(f"  Scale: {scaler_y_dict['scale']}")

In [ ]:
# Create a zip file with all converted files
print("\nCreating zip file...")
!cd tfjs_model && zip -r ../converted_model.zip .
!zip -u converted_model.zip scaler_X.json scaler_y.json
print("converted_model.zip created")

In [ ]:
# Download all files
print("\nDownloading files...")
print("Downloading zip file (contains all files)...")
files.download('converted_model.zip')
print("\nAlso downloading individual files for direct upload...")
files.download('scaler_X.json')
files.download('scaler_y.json')
for file in os.listdir('tfjs_model'):
    files.download(f'tfjs_model/{file}')
print("\n✅ All files downloaded!")

## Next Steps:

1. **Delete old files from Supabase Storage** (important!):
   - Go to: https://supabase.com/dashboard/project/ipfqxgzylmfyffeqpdrb/storage/buckets/models
   - Delete all existing files

2. **Upload new files to Supabase Storage:**
   - Upload these files to the `models` bucket:
     - `model.json`
     - `group1-shard1of1.bin` (or similar .bin files)
     - `scaler_X.json`
     - `scaler_y.json`

3. **Test the edge function** - It will automatically use these files!

**IMPORTANT:** Make sure all files are in the root of the `models` bucket, not in a subfolder.